# Samuel Watkins, 3032132676

# HW 6: Homebrew Computer Vision
## Due Monday Apr 2, 2018 at 2 PM

1. Download the [zip file](https://www.dropbox.com/s/cst9awcjpp08k33/50_categories.tar.gz). Look at some of the images, noting that there are 50 classes in 4244 images (e.g. "goldfish", “llama”, “speed-boat”, ...). Caution: it’s a pretty large file (~208M).
2. Write a set of methods that takes as input one of these images, and then computes real-numbered features as the return. You should produce at least 15 features.
3. Based on the feature set for each image, build a random forest classifier. Produce metrics on your estimated error rates using cross-validation. How much better is this than the expectation with random guessing? What are the 3 most important features?
4. Make sure your final classifier can run on a directory of different images, where a call like `run_final_classifier("/new/directory/path/")` on a directory that contains files like `validation1.jpg`, `validation2.jpg`, etc. will produce an output file that looks like:  
```
filename              predicted_class  
``` 
` `-----------------------------------------------------------------
```
validation1.jpg       unicorn  
validation2.jpg       camel  
```

    We will have a validation set to test how good your classifier is.

# Function to Extract Features from an Image

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from skimage.feature import corner_harris,peak_local_max,canny,corner_peaks,blob_doh
from skimage.segmentation import slic
from skimage.color.colorconv import rgb2grey,grey2rgb
from skimage.filters import frangi,gaussian,threshold_li
from skimage.transform import rescale,hough_line,hough_line_peaks
from skimage.measure import shannon_entropy

In [2]:
def extractImageFeatures(pathToImage,smallImageSize=16000):
    imageArray = plt.imread(pathToImage).astype("float")
    if len(imageArray.shape)<3:
        imageArray = grey2rgb(imageArray)
    greyImgArr = rgb2grey(imageArray)
    scaleFactor = np.sqrt(smallImageSize/np.prod(greyImgArr.shape))
    imgScaled = rescale(greyImgArr,scaleFactor,mode = "constant")
    imgScaledColor = rescale(imageArray,scaleFactor,mode = "constant")
    imgScaledGauss = gaussian(imgScaled)
    
    # dumb features
    imgSize = np.prod(greyImgArr.shape)
    imgHeight = greyImgArr.shape[0]
    imgWidth = greyImgArr.shape[1]
    aspectRatio = imgWidth/imgHeight
    
    avgAllChans = np.mean(imageArray)
    
    avgRedChan = np.mean(imageArray[:,:,0])
    avgGreenChan = np.mean(imageArray[:,:,1])
    avgBlueChan = np.mean(imageArray[:,:,2])
    
    ratioRedBlue = avgRedChan/avgBlueChan
    ratioBlueGreen = avgBlueChan/avgGreenChan
    ratioRedGreen = avgRedChan/avgGreenChan
    
    # "smart" features
    corners = corner_harris(imgScaled)
    numCorners = len(corner_peaks(corners))
    
    peaks = peak_local_max(imgScaled)
    numPeaks = len(peaks)
    
    segments = slic(imgScaled)
    numSegments = np.max(segments)
    
#     blobs = blob_doh(imgScaledGauss)
#     numBlobs = len(blobs)
    
    edges = canny(frangi(imgScaledGauss))
    edgeLength = np.sum(edges)
    if edgeLength == 0:
        edgeLength+=1
        
    edgesZoomed = edges[int(imgScaled.shape[0]/4):-int(imgScaled.shape[0]/4),
                        int(imgScaled.shape[1]/4):-int(imgScaled.shape[1]/4)]
    edgesInMiddle = np.sum(edgesZoomed)/edgeLength
    
    hspace,angles,distances = hough_line(edges)
    if np.sum(hspace)>0:
        numLines = len(hough_line_peaks(hspace,angles,distances)[0])
    else:
        numLines = 0
    
    ratioLinesEdges = numLines/edgeLength
    
    horizontalEdges = edges[int(edges.shape[0]/2)]
    verticalEdges = edges[:,int(edges.shape[1]/2)]
    horizontalEdgeCrossings = sum(horizontalEdges[:-1]!=horizontalEdges[1:])
    verticalEdgeCrossings = sum(verticalEdges[:-1]!=verticalEdges[1:])

    ratioCornersPeak = numCorners/numPeaks
    ratioCornersSegments = numCorners/numSegments
    ratioCornersEdges = numCorners/edgeLength
    ratioPeaksSegments = numPeaks/numSegments
    ratioPeaksEdges = numPeaks/edgeLength
    ratioSegmentsEdges = numSegments/edgeLength
    
    shanent = shannon_entropy(imageArray)
    
    thresh = threshold_li(imgScaled)
    foreground = imgScaled <=thresh
    foregroundSize = np.sum(foreground)
    
    ratioForegroundEdge = foregroundSize/edgeLength
    
    edgeRatio = np.sum(canny(gaussian(foreground)))/edgeLength
    
    avgForegroundRed = np.mean(imgScaledColor[:,:,0][foreground])
    avgForegroundGreen = np.mean(imgScaledColor[:,:,1][foreground])
    avgForegroundBlue = np.mean(imgScaledColor[:,:,2][foreground])
    
    ratioFGRedGreen = avgForegroundRed/avgForegroundGreen
    ratioFGRedBlue = avgForegroundRed/avgForegroundBlue
    ratioFGGreenBlue = avgForegroundGreen/avgForegroundBlue

    features=np.array([imgSize,imgHeight,imgWidth,aspectRatio,ratioRedBlue,ratioBlueGreen,
                        ratioRedGreen,numPeaks,numSegments,edgeLength,
                        horizontalEdgeCrossings,verticalEdgeCrossings,
                        ratioCornersPeak,ratioCornersSegments,ratioPeaksSegments,
                        ratioPeaksEdges,ratioSegmentsEdges,shanent,thresh,foregroundSize,
                        avgForegroundRed,avgForegroundGreen,avgForegroundBlue,
                        ratioFGRedGreen,ratioFGRedBlue,ratioFGGreenBlue,numLines,#numBlobs,
                        ratioCornersEdges,avgAllChans,avgRedChan,avgGreenChan,avgBlueChan,
                        edgeRatio,edgesInMiddle,ratioLinesEdges,ratioForegroundEdge])
    
    if np.any(np.isnan(features)):
        print(pathToImage)
    features[np.isnan(features)]=0.0
    features[np.isinf(features)]=0.0
    
    return features

In [ ]:
from skimage.feature import blob_doh,shape_index
from copy import deepcopy
from skimage.filters import try_all_threshold,threshold_minimum
from skimage.color import label2rgb
from skimage.segmentation import random_walker


pathToImage = "/home/sam/Documents/watkins-ay250-s2018-hw/hw_6/50_categories/bear/bear_0012.jpg"
imageArray = plt.imread(pathToImage).astype("float")
imageArray = grey2rgb(imageArray)
greyImgArr = rgb2grey(imageArray)
scaleFactor = np.sqrt(16000.0/np.prod(greyImgArr.shape))
imgScaledColor = rescale(imageArray,scaleFactor,mode = "constant")
imgScaled = rescale(greyImgArr,scaleFactor,mode = "constant")
threshval = threshold_otsu(gaussian(imgScaled))
imgBackground = gaussian(imgScaled) > threshval
imgForeground = deepcopy(imgScaled)
imgForeground[imgBackground] = 0
binary = gaussian(imgScaled) <=threshval
imgFilt = binary
edgePic = canny(frangi(gaussian(imgScaled)))

val = threshold_minimum(imgScaled)
binary = imgScaled <= val
plt.imshow(canny(gaussian(binary)))

plt.imshow(imgScaled[int(imgScaled.shape[0]/4):-int(imgScaled.shape[0]/4),int(imgScaled.shape[1]/4):-int(imgScaled.shape[1]/4)])

# blobs = blob_doh(gaussian(imgForeground),min_sigma=1.0,max_sigma=30,log_scale=False)

# nonEdgeBlobs = np.logical_and.reduce((np.all(blobs!=0,axis=1),blobs[:,0]!=imgScaled.shape[0]-1,blobs[:,1]!=imgScaled.shape[1]-1))

# blobs = blobs[nonEdgeBlobs]
# print(imgScaled.shape)
# print(len(blobs))
# print(blobs)

# fig, ax = plt.subplots(figsize=(9, 3))
# ax.imshow(imgScaled, interpolation='nearest')
# for blob in blobs:
#     y, x, r = blob
#     c = plt.Circle((x, y), r, color='red', linewidth=2, fill=False)
#     ax.add_patch(c)

# Extract Features from All Images

In [3]:
from glob import glob
from sklearn.model_selection import train_test_split
from multiprocessing import Pool
from time import time

In [4]:
pathToImageFolders = "50_categories/"
eachFolder = glob(pathToImageFolders+"*/")
train_size = 0.5 # ratio of training dataset to total dataset
X = list()
Y = list()

# open up 16 processes to extract features in parallel
num_processes = 16
pool = Pool(processes=num_processes)

starttime = time()
for iFolder,folder in enumerate(eachFolder):
    print(f"Looking in folder {iFolder+1} of {len(eachFolder)} folders...")
    filesInFolder = glob(folder+"*.jpg")
    parallelFeatures = pool.map(extractImageFeatures,filesInFolder)
    X.append(np.vstack(parallelFeatures))
    Y.append(np.repeat(folder[len(pathToImageFolders):-1],len(filesInFolder)))

print(time()-starttime)

pool.terminate()
del pool

X = np.vstack(X)
Y = np.concatenate(Y)
        
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,train_size=train_size,stratify=Y)


Looking in folder 1 of 50 folders...
Looking in folder 2 of 50 folders...
Looking in folder 3 of 50 folders...
Looking in folder 4 of 50 folders...
Looking in folder 5 of 50 folders...
Looking in folder 6 of 50 folders...
Looking in folder 7 of 50 folders...
Looking in folder 8 of 50 folders...
Looking in folder 9 of 50 folders...
Looking in folder 10 of 50 folders...
Looking in folder 11 of 50 folders...
Looking in folder 12 of 50 folders...
Looking in folder 13 of 50 folders...
Looking in folder 14 of 50 folders...
Looking in folder 15 of 50 folders...
Looking in folder 16 of 50 folders...
Looking in folder 17 of 50 folders...
Looking in folder 18 of 50 folders...
Looking in folder 19 of 50 folders...
Looking in folder 20 of 50 folders...
Looking in folder 21 of 50 folders...
Looking in folder 22 of 50 folders...
Looking in folder 23 of 50 folders...


/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in long_scalars
/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in long_scalars
/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in long_scalars
/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in long_scalars


Looking in folder 24 of 50 folders...
Looking in folder 25 of 50 folders...
Looking in folder 26 of 50 folders...


/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in long_scalars
/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in long_scalars


Looking in folder 27 of 50 folders...
Looking in folder 28 of 50 folders...
Looking in folder 29 of 50 folders...
Looking in folder 30 of 50 folders...
Looking in folder 31 of 50 folders...


...

/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in long_scalars
/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in long_scalars
/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in long_scalars
/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in long_scalars
/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in long_scalars
/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in long_scalars


Looking in folder 39 of 50 folders...
Looking in folder 40 of 50 folders...
Looking in folder 41 of 50 folders...
Looking in folder 42 of 50 folders...
Looking in folder 43 of 50 folders...
Looking in folder 44 of 50 folders...
Looking in folder 45 of 50 folders...


/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in long_scalars
/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in long_scalars


Looking in folder 46 of 50 folders...
Looking in folder 47 of 50 folders...
Looking in folder 48 of 50 folders...
Looking in folder 49 of 50 folders...
Looking in folder 50 of 50 folders...
350.5047471523285


/home/sam/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


# Build A Random Forest Classifier

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score

In [6]:
randforclf = RandomForestClassifier(n_estimators=50)

randforclf.fit(X_train,Y_train)

pred_rf = randforclf.predict(X_test)

In [10]:
print(f"Score: {metrics.accuracy_score(Y_test,pred_rf)}")
scores = cross_val_score(randforclf,X,Y,cv=5,groups=Y)
print(f"Accuracy from cross-validation: {np.mean(scores)} (+/- {np.std(scores)})")

Score: 0.3067860508953817
Accuracy from cross-validation: 0.3147338692986878 (+/- 0.003915028005286019)


In [8]:
randforclf.feature_importances_

array([0.04037229, 0.04469919, 0.03814068, 0.07204933, 0.02460567,
       0.02620213, 0.02891444, 0.02530363, 0.02546769, 0.02443762,
       0.02041685, 0.01824054, 0.0252138 , 0.02237858, 0.02505821,
       0.02486793, 0.02671337, 0.03635504, 0.02287981, 0.02325356,
       0.02489279, 0.02615189, 0.02466265, 0.02455391, 0.0250627 ,
       0.02474401, 0.02805788, 0.02352151, 0.02212818, 0.02407797,
       0.02118259, 0.02323303, 0.02910648, 0.02620968, 0.03223244,
       0.02461196])

# Compare to Random Guessing

In [ ]:
from sklearn.dummy import DummyClassifier

In [ ]:
dummyclf = DummyClassifier(strategy="prior",random_state=42)

dummyclf.fit(X_train,Y_train)

dummypred_rf = dummyclf.predict(X_test)

In [ ]:
print(metrics.accuracy_score(Y_test,dummypred_rf))
scores = cross_val_score(dummyclf,X,Y,cv=5,groups=Y)
print(f"{np.mean(scores)} (+/- {np.std(scores)})")